In [1]:
# importing modules
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [2]:
csv_list = glob("./dataFiles/*.csv")
csv_list

['./dataFiles\\bom.movie_gross.csv',
 './dataFiles\\name.basics.csv',
 './dataFiles\\title.akas.csv',
 './dataFiles\\title.basics.csv',
 './dataFiles\\title.crew.csv',
 './dataFiles\\title.principals.csv',
 './dataFiles\\title.ratings.csv',
 './dataFiles\\tmdb.movies.csv',
 './dataFiles\\tn.movie_budgets.csv']

In [3]:
cleaned_filenames = [filename.split('\\')[1].replace('.csv', '').replace('.', '_')
                     for filename in csv_list]

In [4]:
orig_dfs = {}
for idx, file in enumerate(csv_list):
    orig_dfs.update({cleaned_filenames[idx]: pd.read_csv(file)})

In [5]:
orig_dfs['bom_movie_gross'].head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


Now that we have all the files loaded into dataframes in an easily accessible structure, lets investigate them all to see what information we have available and make connections between the data included in each file. Besides, we can see what modifications we may need to make to data types, to account for missing values, etc. Going forward we will get a lot of  information to keep track of, so I will make a separate excel doc for use as a reference / table schema to relate these dataframes in the future. 

In [6]:
for item in orig_dfs.items():
        print("===============New DF===============")
        print(f'Title:  {item[0]}')
        print(item[1].info())
        print("================End DF===============")

===============New DF===============
Title:  bom_movie_gross
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
================End DF===============
===============New DF===============
Title:  name_basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   fl

So far, we can notice that, based on column names, there are some potential connections between the different dataframes. Additionally, there are columns that are hard to interpret from their labels, and there are some missing a large majority of values. Lastly, we can note that there are columns with data types that might be undesirable, such as gross columns stored as strings, and date columns stored as objs or ints/floats. First, to get a better understanding of what is in each column, lets look at the head of each dataframe (since print(df.head()) doesn't look as nice in the notebook, create cell for each one instead).

In [7]:
orig_dfs['bom_movie_gross'].head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [8]:
orig_dfs['name_basics'].head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [9]:
orig_dfs['title_akas'].head()

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.0
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.0


In [10]:
orig_dfs['title_basics'].head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [11]:
orig_dfs['title_crew'].head()

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [12]:
orig_dfs['title_principals'].head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [13]:
orig_dfs['title_ratings'].head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [14]:
orig_dfs['tmdb_movies'].head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [36]:
orig_dfs['tn_movie_budgets'].head(10)

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
5,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
6,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200"
7,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,"$300,000,000","$309,420,425","$963,420,425"
8,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209"
9,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923"


Seeing these, there are some obvious joins / connections to make on things we can be nearly confident in referencing the same thing, such as on identifiers tconst and nconst. There are some columns we are somewhat confident match but will have to be careful about, such as the different movie title columns, and some columns we are not sure yet referencing the same movie (or have equivalent values if they do) but potentially could be, such as the ratings and quoted domestic and foreign gross values. Lastly, there are some extra indexing columns we can drop, such as unnamed and id in the last two tables. The ordering columns as well perhaps, as they seem to index rows for multiple records for the same title. Let's get rid of cluttering rows we know aren't useful first. Then we can look at combining dataframes / eliminating other columns.

In [16]:
orig_dfs['tmdb_movies'].drop(columns = ['Unnamed: 0'], inplace = True)
orig_dfs['tn_movie_budgets'].drop(columns = ['id'], inplace = True)
orig_dfs['title_principals'].drop(columns = ['ordering'], inplace = True)
orig_dfs['title_akas'].drop(columns = ['ordering'], inplace = True)

Now, let's look at other problem columns and fix them up. Starting with bom_movie_gross and moving down the line, we see that in that dataframe there is a studio column that could be more interpretable, since they are currently abbreviations. Let's first see what all the unique abbreviations are.

In [35]:
orig_dfs['bom_movie_gross'].studio.unique()

array(['BV', 'WB', 'P/DW', 'Sum.', 'Par.', 'Uni.', 'Fox', 'Wein.', 'Sony',
       'FoxS', 'SGem', 'WB (NL)', 'LGF', 'MBox', 'CL', 'W/Dim.', 'CBS',
       'Focus', 'MGM', 'Over.', 'Mira.', 'IFC', 'CJ', 'NM', 'SPC', 'ParV',
       'Gold.', 'JS', 'RAtt.', 'Magn.', 'Free', '3D', 'UTV', 'Rela.',
       'Zeit.', 'Anch.', 'PDA', 'Lorb.', 'App.', 'Drft.', 'Osci.', 'IW',
       'Rog.', nan, 'Eros', 'Relbig.', 'Viv.', 'Hann.', 'Strand', 'NGE',
       'Scre.', 'Kino', 'Abr.', 'CZ', 'ATO', 'First', 'GK', 'FInd.',
       'NFC', 'TFC', 'Pala.', 'Imag.', 'NAV', 'Arth.', 'CLS', 'Mont.',
       'Olive', 'CGld', 'FOAK', 'IVP', 'Yash', 'ICir', 'FM', 'Vita.',
       'WOW', 'Truly', 'Indic.', 'FD', 'Vari.', 'TriS', 'ORF', 'IM',
       'Elev.', 'Cohen', 'NeoC', 'Jan.', 'MNE', 'Trib.', 'Rocket',
       'OMNI/FSR', 'KKM', 'Argo.', 'SMod', 'Libre', 'FRun', 'WHE', 'P4',
       'KC', 'SD', 'AM', 'MPFT', 'Icar.', 'AGF', 'A23', 'Da.', 'NYer',
       'Rialto', 'DF', 'KL', 'ALP', 'LG/S', 'WGUSA', 'MPI', 'RTWC', 'FIP

Okay, well, there's alot. Most or some of them probably do not exist anymore.. maybe we will wait on this.

A useful discovery from the The Movie Database API page for turning genre_id  to interpretable strings in that table:
MOVIES
Action          28
Adventure       12
Animation       16
Comedy          35
Crime           80
Documentary     99
Drama           18
Family          10751
Fantasy         14
History         36
Horror          27
Music           10402
Mystery         9648
Romance         10749
Science Fiction 878
TV Movie        10770
Thriller        53
War             10752
Western         37

We could use this to replace the genre ID's with text.

Now, we may be able to condense the number of tables while leaving appropriate opportunity for joins. We just need to start checking where records are truly pointing toward the same information, then bringing information together. For example, we can imagine for a unique movie, the following information would be useful to collect for generating visualizations: tconst(table primary key), primary_title (for display purposes), genre(s), release_year, run_time, production_studio, production_budget, domestic_gross, domestic_ROI, worldwide_gross, worldwide_ROI. We can collect the rest of the useful data in two other tables probably. Brainstorming, I would be interested in one table having columns tconst, rating, num_votes and popularity, and a second table having tconst, and then columns for each crew job type (producer, director, writer, actor, etc.) containing lists of the crew of that type for that film. Let's look at the sizes of each dataframe to see if there are ones we can join to / build off of while retaining the maximum amount of information.

In [17]:
for item in orig_dfs.items():
        print("===============New DF===============")
        print(f'Title:  {item[0]}')
        print(item[1].shape)
        print("================End DF===============")

===============New DF===============
Title:  bom_movie_gross
(3387, 5)
================End DF===============
===============New DF===============
Title:  name_basics
(606648, 6)
================End DF===============
===============New DF===============
Title:  title_akas
(331703, 7)
================End DF===============
===============New DF===============
Title:  title_basics
(146144, 6)
================End DF===============
===============New DF===============
Title:  title_crew
(146144, 3)
================End DF===============
===============New DF===============
Title:  title_principals
(1028186, 5)
================End DF===============
===============New DF===============
Title:  title_ratings
(73856, 3)
================End DF===============
===============New DF===============
Title:  tmdb_movies
(26517, 9)
================End DF===============
===============New DF===============
Title:  tn_movie_budgets
(5782, 5)
================End DF===============


So we are seeing that we have data for many more movies than we have budget information for. Since we are trying to produce profitable movies, being able to determine indicators driving profit is necessary, unless we can become very confident that something is an effective proxy for profit such as rating or popularity. It is possible that between the tables with budgetary information, i.e. 'bom_movie_gross' and 'tn_movie_budgets' we can cover more movies than either alone, and hopefully fill out some of the other information with the other tables we have. However, we only have the budget information for those from the 'tn_movie_budgets' table, which means we can only calculate ROI's for movies from that table, unless we can find the information elsewhere OR are happy filling in values for NULLs. We can look into that but it is hard to imagine that budget can easily be predicted and its assigned value has a huge impact on ROI.

Upon doing some research, I have determined by looking at lists on the The Numbers and The Movie Database websites that the domestic, foreign, and worldwide gross numbers from each site are close enough that they can be considered the same. Additionally, the 'worldwide_gross' from the 'tn_movie_budgets' table is the same as the sum of the 'domestic' and 'foreign_gross' columns of the 'bom_movie_gross' table, so we can make an aggregate column in the Box Office Mojo table and also produce a 'foreign_gross' column for the The Numbers table. Down the line, it may be possible to scrape The Numbers or Box Office Mojo for budgetary information for more movies.

Additionally, there is a way to query TMDB using an IMDB tconst through a python module called tmdbsimple:

import tmdbsimple as tmdb
tmdb.API_KEY = '<your api key>'
response = tmdb.Find('tt0266543').info(external_source='imdb_id')
response

{'movie_results': [{'id': 12,
   'video': False,
   'vote_count': 13478,
   'vote_average': 7.8,
   'title': 'Finding Nemo',
   'release_date': '2003-05-30',
   'original_language': 'en',
   'original_title': 'Finding Nemo',
   'genre_ids': [16, 10751],
   'backdrop_path': '/dFYguAfeVt19qAbzJ5mArn7DEJw.jpg',
   'adult': False,
   'overview': "Nemo, an adventurous young clownfish, is unexpectedly taken from his Great Barrier Reef home to a dentist's office aquarium. It's up to his worrisome father Marlin and a friendly but forgetful fish Dory to bring Nemo home -- meeting vegetarian sharks, surfer dude turtles, hypnotic jellyfish, hungry seagulls, and more along the way.",
   'poster_path': '/xVNSgrsvpcAHPnyKf2phYxyppNZ.jpg',
   'popularity': 31.27}],
 'person_results': [],
 'tv_results': [],
 'tv_episode_results': [],
 'tv_season_results': []}
    
Or, simply using a get request you can get the above info and more, seen at https://developers.themoviedb.org/3/movies/get-movie-details. 
    
It may be worth looking into that package or otherwise scraping the web API to get a complete dataset from one source.


Anyway, let's begin by joining the two tables we have with budgetary information and see what we get. 

In [27]:
budget_merge = orig_dfs['tn_movie_budgets'].merge(orig_dfs['bom_movie_gross'],
                                                  how='right',
                                                  left_on='movie',
                                                  right_on='title')
budget_merge.head(20)

,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,title,studio,domestic_gross_y,foreign_gross,year
0,"Jun 18, 2010",Toy Story 3,"$200,000,000","$415,004,880","$1,068,879,522",Toy Story 3,BV,415000000.0,652000000,2010
1,NaN,NaN,NaN,NaN,NaN,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,NaN,NaN,NaN,NaN,NaN,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,"Jul 16, 2010",Inception,"$160,000,000","$292,576,195","$835,524,642",Inception,WB,292600000.0,535700000,2010
4,"May 21, 2010",Shrek Forever After,"$165,000,000","$238,736,787","$756,244,673",Shrek Forever After,P/DW,238700000.0,513900000,2010
5,"Jun 30, 2010",The Twilight Saga: Eclipse,"$68,000,000","$300,531,751","$706,102,828",The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,"May 7, 2010",Iron Man 2,"$170,000,000","$312,433,331","$621,156,389",Iron Man 2,Par.,312400000.0,311500000,2010
7,"Nov 24, 2010",Tangled,"$260,000,000","$200,821,936","$586,477,240",Tangled,BV,200800000.0,391000000,2010
8,"Jul 9, 2010",Despicable Me,"$69,000,000","$251,513,985","$543,464,573",Despicable Me,Uni.,251500000.0,291600000,2010
9,"Mar 26, 2010",How to Train Your Dragon,"$165,000,000","$217,581,232","$494,870,992",How to Train Your Dragon,P/DW,217600000.0,277300000,2010


In [28]:
budget_merge.shape

(3396, 10)

To be honest, after looking at this stuff, it is very tempting to take the tmdb movies dataframe and use those IDs or otherwise generate a completely new and much larger dataset from TMDB. It can result in having budget information for way more movies. However, I notice that their budget estimates are quit low compared to The Numbers estimates, and seeing as both are really guesses and are user maintained / reported it is hard to know which to be more confident about. So long as we are consistent, it should be fine. I think, as opposed to replacing the entire dataset, we can use their API to get a few columns of additional information for the tmdb_movies dataframe, i.e. their imdb ID number (eg. tt######) for connecting reliably with other tables, and the listed budget and revenue for each film, since that will solve the problem of having to look at an intersection of the information collected from Box Office Mojo and The Numbers.